In [ ]:
import os
import datetime
from tqdm import tqdm
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from championWrapper import ChampionWrapper

In [ ]:
in_data = pd.read_csv('pb_table.csv', index_col=False)
champs = ChampionWrapper()

num_champs = len(champs.champToID)
num_champs


In [ ]:
#isolate the input frames of hero choices by name
isolated_input = in_data[['BB1', 'RB1', 'BB2', 'RB2', 'BB3', 'RB3', 'BP1', 'RP1','RP2','BP2','BP3', 'RP3', 'RB4','BB4', 'RB5', 'BB5', 'RP4', 'BP4','BP5', 'RP5']]
isolated_input.head()

In [ ]:
#Dataframe to Numpy
inputnpArray = isolated_input.to_numpy()
inputnpArray


In [ ]:
# Numpy convert Names to Ids
for match in range(len(inputnpArray)):
    for column in range(len(inputnpArray[match])):
        inputnpArray[match][column] = champs.cNameToId(inputnpArray[match][column])

inputnpArray



Here we have the data   
In Raw form we want to make it into [In][Label] sets to    
use in our Sequence to Sequence model   

In [ ]:
#Here we have the data
# In Raw form we want to make it into [In][Label] sets to 
# use in our Sequence to Sequence model

In [ ]:
numberpd = pd.DataFrame(inputnpArray, columns=['BB1', 'RB1', 'BB2', 'RB2', 'BB3', 'RB3', 'BP1', 'RP1','RP2','BP2','BP3', 'RP3', 'RB4','BB4', 'RB5', 'BB5', 'RP4', 'BP4','BP5', 'RP5'])
print(numberpd)

In [ ]:
traindf, testdf = train_test_split(numberpd, test_size=0.2)
testdf, valdf = train_test_split(testdf, test_size=0.5)

In [ ]:
tfl = tf.keras.layers
n_features=num_champs
results = {}
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5)
the_loss = 'categorical_crossentropy'
num_epochs=500
r_state = 20
tf.random.set_seed=r_state
from WindowGenerator import SplitGen

splitter = SplitGen()
train, test = train_test_split(inputnpArray, test_size=0.2, random_state=r_state)
test, val = train_test_split(test, test_size=0.5, random_state=r_state)

i=18
o=2
f=num_champs

X_train, y_train = splitter.split_sequences(train, i, o, f)
X_test, y_test = splitter.split_sequences(test, i, o, f)
X_val, y_val = splitter.split_sequences(val, i, o, f)






In [ ]:
lSTM_model = tf.keras.Sequential([
    tfl.LSTM(512, activation='relu', input_shape=(i, f)),
    tfl.RepeatVector(o),
    tfl.Dense(64),
    tfl.Dense(f, activation='softmax')
])
lSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

lSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['lstm']=lSTM_model.evaluate(X_test, y_test)
print(results['lstm'])

In [ ]:
model=lSTM_model
print(model.evaluate(X_train, y_train))
print(model.evaluate(X_val, y_val))
print(model.evaluate(X_test, y_test))

In [ ]:
rnn_model = tf.keras.Sequential([
    tfl.SimpleRNN(512, input_shape=(i, f)),
    tfl.Dense(128),
    tfl.Dense(f, activation='softmax'),
    tfl.RepeatVector(o)

    
])
rnn_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

rnn_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['rnn']=rnn_model.evaluate(X_test, y_test)
print(results['rnn'])

In [ ]:
model=rnn_model
print(model.evaluate(X_train, y_train))
print(model.evaluate(X_val, y_val))
print(model.evaluate(X_test, y_test))

In [ ]:
bi_LSTM_model = tf.keras.Sequential([
    tfl.Bidirectional(tfl.LSTM(512, activation='relu', input_shape=(i, f))),
    tfl.RepeatVector(o),
    tfl.Dropout(0.2),
    tfl.Dense(256, activation='relu'),
    tfl.Dropout(0.2),
    tfl.TimeDistributed(tfl.Dense(256)),
    tfl.Dropout(0.4),
    tfl.TimeDistributed(tfl.Dense(f, activation='softmax'))


])
bi_LSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

bi_LSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es],verbose=1, epochs=num_epochs)
print("\n\neval-----")
results['bilstm']=bi_LSTM_model.evaluate(X_test, y_test)
print(results['bilstm'])



In [ ]:
model=bi_LSTM_model
print(model.evaluate(X_train, y_train))
print(model.evaluate(X_val, y_val))
print(model.evaluate(X_test, y_test))

In [ ]:
print(results['bilstm'])

In [ ]:
CONV_WIDTH = 3
conv_model = tf.keras.Sequential([

 tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(32, activation='relu', kernel_size=(CONV_WIDTH)),
    tf.keras.layers.Flatten(),
    tfl.Dense(128),
    tf.keras.layers.Dense(o*f,
                          kernel_initializer=tf.initializers.zeros(), activation='softmax'),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([o, f])

])
conv_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

conv_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----\n")
results['conv']=conv_model.evaluate(X_test, y_test)
print(results['conv'])

In [ ]:
model=conv_model
print(model.evaluate(X_train, y_train))
print(model.evaluate(X_val, y_val))
print(model.evaluate(X_test, y_test))

In [ ]:
clstm_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, activation='relu', kernel_size=(CONV_WIDTH)),
    tfl.MaxPooling1D(pool_size=2),
    tfl.Flatten(),
    tfl.RepeatVector(o),
    tfl.LSTM(512, activation='relu', return_sequences=True), 
    tfl.TimeDistributed(tfl.Dense(256,activation='relu')),
    tfl.TimeDistributed(tfl.Dense(f))
])

clstm_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

clstm_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\neval-----\n")
results['clstm']=clstm_model.evaluate(X_test, y_test)
print(results['clstm'])

In [ ]:
print(results)
win = max(results, key=lambda x: results[x][1])
print(f'max on test: {win}: {results[win]}')

In [ ]:
models = [clstm_model, bi_LSTM_model, lSTM_model,hydoubleLSTM_model,doubleLSTM_model,bidoubleLSTM_model,rnn_model,singrnn_model,conv_model]
losses = ['mse', 'categorical_crossentropy', 'binary_crossentropy']
names = ['clstm', 'bilstm', 'lstm', 'hylstm', 'doubllstm', 'bidoublels', 'rnn', 'singrnn', 'conv']
results = {}
for loss in losses:
    the_loss = loss
    print(loss)
    for model, name in tqdm(zip(models, names)):
        print(name)
        model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])
        model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs, verbose=0)
        results[f'{loss}_{name}']= model.evaluate(X_test, y_test)




In [ ]:
print(results)
win = max(results, key=lambda x: results[x][1])
print(f'max on test: {win}: {results[win]}')

In [ ]:
models = [clstm_model, bi_LSTM_model, lSTM_model,hydoubleLSTM_model,doubleLSTM_model,bidoubleLSTM_model,rnn_model,singrnn_model,conv_model]
losses = ['mse', 'categorical_crossentropy', 'binary_crossentropy']
names = ['clstm', 'bilstm', 'lstm', 'hylstm', 'doubllstm', 'bidoublels', 'rnn', 'singrnn', 'conv']
for loss in losses:
    the_loss = loss
    print(loss)
    for model, name in tqdm(zip(models, names)):
        print(name)
        model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])
        model.fit(X_train,y_train, validation_data=(X_val, y_val), epochs=num_epochs, verbose=0)
        results[f'long_{loss}_{name}']= model.evaluate(X_test, y_test)

In [ ]:
print(results)
win = max(results, key=lambda x: results[x][1])
print(f'max on test: {win}: {results[win]}')